# Time Period: 20140630-20140930


In [1]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline

### X1: BE/ME Ratio in 2014 (same throughout the year)

In [29]:
AShareEODDerivativeIndicator = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Supplementary datasets/EODDerivativeIndicator.csv", header=0)

In [2]:
BEME_ratio_df_2014 = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Intermediate Files/BEME_ratio_df_2014.csv", header=0)

In [3]:
norm_BEME_ratio_list = list(BEME_ratio_df_2014['norm_BEME_ratio'])# 268 companies available total

### X2: Last Quarter Return: 2014 Q2

Duration: 20140331 - 20140630

return = [SP(20140630) - SP(20140331)]/SP(20140331)

sample pool: companies in BEME_ratio_df['WINDCODE']

#### a) Last Quarterly Return for Individual Company

In [4]:
##### Load in all the files of Stock Price

# set current working directory to the directory storing stock price data
os.chdir('/Users/yiliwang/Documents/ACR Internship/data/')

In [5]:
files = [file for file in os.listdir( os.curdir ) if file.endswith(".csv")]

In [6]:
# Store files to dictionary using their own name
frames = {}
for each_file in files:
    frames[os.path.splitext(each_file)[0]] = pd.read_csv(each_file)

##### Extract Stock Price Data on Starting Day and End Day

In [7]:
# Obatin Close Price
start_close = [None]*len(BEME_ratio_df_2014['WINDCODE'])
end_close = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [8]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    company = BEME_ratio_df_2014['WINDCODE'][i]
    start_close[i] =float(frames[company][frames[company]['TRADE_DT'] == 20140331]['S_DQ_ADJCLOSE'])
    end_close[i] = float(frames[company][frames[company]['TRADE_DT'] == 20140630]['S_DQ_ADJCLOSE'])

In [9]:
# create an empty list to store return for individual company
com_qtr_return = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [10]:
# Calculate quarterly return
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    com_qtr_return[i] = (end_close[i] - start_close[i])/start_close[i]

#### b) Last Quarterly Return for CSI300 Index

In [11]:
csi300_IndexData= pd.read_csv("/Users/yiliwang/Documents/ACR Internship/csi_300_IndexData_daily_.csv", header=0)

In [12]:
index_start = float(csi300_IndexData[csi300_IndexData['date'] == '2014-03-31']['last_price']) # index price on start day
index_end = float(csi300_IndexData[csi300_IndexData['date'] == '2014-06-30']['last_price'])  # index price on end day

In [13]:
index_qtr_return = (index_end - index_start)/index_start # calculate the index quarterly return

In [14]:
print index_qtr_return

0.00649408125521


#### Difference = Company Return - Index Return 
Result List: norm_diff_csi300_LAST

In [15]:
diff_return_com_csi300 = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [16]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    diff_return_com_csi300[i] = com_qtr_return[i] - index_qtr_return

In [17]:
# convert into a dataframe
Weighted_Return = pd.DataFrame({'Company':BEME_ratio_df_2014['WINDCODE'],'diff_csi300_LAST': diff_return_com_csi300})

In [18]:
# Normalize Function
def normalize_list(input_list):
    empty_list = [None]*len(input_list)
    for i in range(len(input_list)):
        empty_list[i] = (input_list[i]-input_list.mean())/input_list.std()
    return empty_list

In [19]:
Weighted_Return['norm_diff_csi300_LAST'] = normalize_list(Weighted_Return['diff_csi300_LAST'])

In [20]:
Weighted_Return.to_csv("LAST_Weighted_Return_2014_Q3.csv")

###### X2: norm_diff_csi300_LAST

### 3. AShareEODDerivativeIndicator['FLOAT_A_SHR_TODAY']
total free-float shares per day * unadjusted close price

In [21]:
# Extract trading dates of certain period 
dates_trade = frames['000001.SZ']['TRADE_DT'].unique()
dates_trade.sort() # sort the date

In [22]:
start_dateind = list(dates_trade).index(20140630) # index of start date
end_dateind = list(dates_trade).index(20140930) # index of end date

In [23]:
# target_dates: list of trading dates in that period
target_dates = dates_trade[start_dateind:end_dateind+1]

In [24]:
target_dates

array([20140630, 20140701, 20140702, 20140703, 20140704, 20140707,
       20140708, 20140709, 20140710, 20140711, 20140714, 20140715,
       20140716, 20140717, 20140718, 20140721, 20140722, 20140723,
       20140724, 20140725, 20140728, 20140729, 20140730, 20140731,
       20140801, 20140804, 20140805, 20140806, 20140807, 20140808,
       20140811, 20140812, 20140813, 20140814, 20140815, 20140818,
       20140819, 20140820, 20140821, 20140822, 20140825, 20140826,
       20140827, 20140828, 20140829, 20140901, 20140902, 20140903,
       20140904, 20140905, 20140909, 20140910, 20140911, 20140912,
       20140915, 20140916, 20140917, 20140918, 20140919, 20140922,
       20140923, 20140924, 20140925, 20140926, 20140929, 20140930])

In [30]:
def find_FAST_mean(company): # For a given company, calculate mean of FLOAT_A_SHR_TODAY *close price (unadjusted) on trading days
    FAST_product = [None]*len(target_dates)
    
    for i in range(len(target_dates)):
        close_p = frames[company][frames[company]['TRADE_DT'] == target_dates[i]]['S_DQ_CLOSE'].values # get close price on a particular date
        
        # dataframe of the company
        df_company = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['S_INFO_WINDCODE'] ==company]
        FAST_day = float(df_company[df_company['TRADE_DT']==target_dates[i]]['FLOAT_A_SHR_TODAY']) #FLOAT_A_SHR_TODAY of a particular day
        
        FAST_product[i] = FAST_day*close_p
        
    FAST_mean = np.mean(FAST_product)
    return FAST_mean

In [31]:
FAST_mean_list = [find_FAST_mean(company) for company in BEME_ratio_df_2014['WINDCODE']]

In [32]:
FAST_mean_df = pd.DataFrame({'Company':BEME_ratio_df_2014['WINDCODE'],'FAST_mean':FAST_mean_list})

In [33]:
FAST_mean_df['FAST_mean_norm'] = normalize_list(FAST_mean_df['FAST_mean'])

In [34]:
FAST_mean_df

,Company,FAST_mean,FAST_mean_norm
0,002475.SZ,2.475450e+06,-0.178524
1,002470.SZ,1.214770e+06,-0.287127
2,600036.SH,2.173103e+07,1.480268
3,002146.SZ,1.744583e+06,-0.241485
4,601988.SH,5.207636e+07,4.094398
5,000895.SZ,4.217974e+06,-0.028412
6,600048.SH,6.026765e+06,0.127408
7,000718.SZ,7.219400e+05,-0.329582
8,002304.SZ,5.228155e+06,0.058611
9,600031.SH,4.182176e+06,-0.031496


In [35]:
FAST_mean_df.to_csv("FAST_mean_2014_Q3.csv") # save it as csv

##### X3: FAST_mean_df['FAST_mean_norm']

## Y : Current Quarter Return

Duration: 20140930 - 20141231

return = [SP(20141231) - SP(20140930)]/SP(20140930)

sample pool: companies in BEME_ratio_df['WINDCODE']

In [36]:
# Obatin Close Price on starting and end dates for current qtr
start_close_c = [None]*len(BEME_ratio_df_2014['WINDCODE'])
end_close_c = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [37]:
# Dataframe start_close: start_close price for 268 companies

for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    company = BEME_ratio_df_2014['WINDCODE'][i]
    start_close_c[i] =float(frames[company][frames[company]['TRADE_DT'] == 20140630]['S_DQ_ADJCLOSE'])
    end_close_c[i] = float(frames[company][frames[company]['TRADE_DT'] ==20140930]['S_DQ_ADJCLOSE'])

In [38]:
# create an empty list to store return for individual company for current qtr
com_qtr_return_CURRENT = [None]*len(BEME_ratio_df_2014['WINDCODE']) 

In [39]:
# Calculate quarterly return
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    com_qtr_return_CURRENT[i] = (end_close_c[i] - start_close_c[i])/start_close_c[i]

In [40]:
index_start_c = float(csi300_IndexData[csi300_IndexData['date'] == '2014-06-30']['last_price']) 
index_end_c = float(csi300_IndexData[csi300_IndexData['date'] == '2014-09-30']['last_price']) 

In [41]:
index_qtr_return_CURRENT = (index_end_c - index_start_c)/index_start_c # calculate the index quarterly return

In [42]:
index_qtr_return_CURRENT

0.13150135847554495

In [43]:
diff_return_com_csi300_CURRENT = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [44]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    diff_return_com_csi300_CURRENT[i] = com_qtr_return_CURRENT[i] - index_qtr_return_CURRENT

In [45]:
diff_return_com_csi300_CURRENT[0:10]

[-0.1324534803471444,
 0.16516867179690709,
 -0.05155148378882804,
 -0.008076863826317712,
 -0.07122738587280517,
 -0.19381269957084465,
 -0.012601726799670199,
 0.09991128092965953,
 0.11242858213935386,
 0.05356095659663282]

In [46]:
Y_2014_Q3 = pd.DataFrame({'Company':BEME_ratio_df_2014['WINDCODE'],'Y':diff_return_com_csi300_CURRENT})

In [47]:
Y_2014_Q3.to_csv("Y_2014_Q3.csv")

##### Y: diff_return_com_csi300_CURRENT

In [48]:
TFM_2014_Q3 = pd.DataFrame({'X_1':norm_BEME_ratio_list,'X_2':Weighted_Return['norm_diff_csi300_LAST'], 'X_3': FAST_mean_df['FAST_mean_norm'], 'Y':diff_return_com_csi300_CURRENT})

In [ ]:
TFM_2014_Q3[:10]

In [50]:
TFM_2014_Q3.to_csv("TFM_2014_Q3.csv")

In [51]:
def outlier_index(list_name, std_num):
    mean = list_name.mean() # mean
    std = np.std(list_name, axis=0) # standard deviation
    
    upper_b = mean+std_num*std
    lower_b = mean-std_num*std
    
    ind_upper = [i for i in range(len(list_name)) if list_name[i] > upper_b]
    ind_lower = [i for i in range(len(list_name)) if list_name[i] < lower_b]
    outlier_index = ind_upper+ind_lower
    
    return outlier_index

In [52]:
outlier_all =outlier_index(TFM_2014_Q3['X_1'], 5) + outlier_index(TFM_2014_Q3['X_2'], 5)+ outlier_index(TFM_2014_Q3['X_3'], 5)+ outlier_index(TFM_2014_Q3['Y'], 5)

In [53]:
TFM_2014_Q3_NO = TFM_2014_Q3.drop(TFM_2014_Q3.index[outlier_all])

In [55]:
TFM_2014_Q3_NO[:10]

,X_1,X_2,X_3,Y
0,-0.943715,1.944860,-0.178524,-0.132453
1,-0.663215,-0.229808,-0.287127,0.165169
2,0.831511,0.006560,1.480268,-0.051551
3,0.060847,-2.130893,-0.241485,-0.008077
4,1.576604,0.158118,4.094398,-0.071227
5,-0.981901,-0.786710,-0.028412,-0.193813
6,1.036689,-0.199926,0.127408,-0.012602
7,-0.199968,0.078030,-0.329582,0.099911
8,-0.417362,0.240642,0.058611,0.112429
9,-0.118337,-1.111186,-0.031496,0.053561


In [56]:
TFM_2014_Q3_NO.to_csv("TFM_2014_Q3_NO.csv")

# ************* Result DataFrame: TFM_2014_Q3_NO.csv *************